In [1]:
from math import sin

In [2]:
def MyHash(s):
    A = 0x464c4147
    B = 0x7b754669
    C = 0x6e645468
    D = 0x65456173
    E = 0x74657245
    F = 0x6767217D
    def G(X,Y,Z):
        return (X ^ (~Z | ~Y) ^ Z) & 0xFFFFFFFF
    def H(X,Y):
        return (X << Y | X >> (32 - Y)) & 0xFFFFFFFF
    X = [int((0xFFFFFFFE) * sin(i)) & 0xFFFFFFFF for i in range(256)]
    s_size = len(s)
    s += bytes([0x80])
    if len(s) % 128 > 120:
        while len(s) % 128 != 0: s += bytes(1)
    while len(s) % 128 < 120: s += bytes(1)
    # May pad to 120 bytes or 128 + 120 bytes
    print(b"After Padding: " + s)
    print("Length: " + str(len(s)), end="\n\n")
    # Add 8 Byte Message Length (in Bits)
    s += bytes.fromhex(hex(s_size * 8)[2:].rjust(16, '0'))
    print(b"Add Length in Bits: " + bytes.fromhex(hex(s_size * 8)[2:].rjust(16, '0')))
    print("Final Length: " + str(len(s)))
    print(b"Final Str: " + s)
    # Final Str => 128 Bytes * k
    for i, b in enumerate(s):
        # 32 Bytes a Block
        k, l = int(b), i & 0x1f
        A = (B + H(A + G(B,C,D) + X[k], l)) & 0xFFFFFFFF
        B = (C + H(B + G(C,D,E) + X[k], l)) & 0xFFFFFFFF
        C = (D + H(C + G(D,E,F) + X[k], l)) & 0xFFFFFFFF
        D = (E + H(D + G(E,F,A) + X[k], l)) & 0xFFFFFFFF
        E = (F + H(E + G(F,A,B) + X[k], l)) & 0xFFFFFFFF
        F = (A + H(F + G(A,B,C) + X[k], l)) & 0xFFFFFFFF
    return ''.join(map(lambda x : hex(x)[2:].rjust(8, '0'), [A, F, C, B, D, E]))

In [3]:
user = "HelloLEA"
password = "LEA"
# Random Can't Guess
session_id = "3f4cb81e003da15b029d"

target = "&&".join([user, password, session_id]).encode()
print(len(target))
MyHash(target)

35
b'After Padding: HelloLEA&&LEA&&3f4cb81e003da15b029d\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Length: 120

b'Add Length in Bits: \x00\x00\x00\x00\x00\x00\x01\x18'
Final Length: 128
b'Final Str: HelloLEA&&LEA&&3f4cb81e003da15b029d\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x18'


'893b9d2a40e1aba15f3a43291825a23d5cab8b37e762ec35'

In [4]:
hex(35 * 8)

'0x118'

In [5]:
# Start to Crack the LEA!!
user = "Admin"
# No Password
session = "aa27d727b998940c7879"
mac = "b18b653eb810996de33b084741dbdd9e11af3b4811c5253e"

# Extract Last Vector

_seq = ["A", "F", "C", "B", "D", "E"]
vector = []

for i in range(6):
    print(_seq[i] + ": " + mac[i*8: (i+1)*8])
    vector.append(int(mac[i*8: (i+1)*8], 16))

print(vector)
# username&&password&&session&&old-padding && cmd

A: b18b653e
F: b810996d
C: e33b0847
B: 41dbdd9e
D: 11af3b48
E: 11c5253e
[2978702654, 3088095597, 3812296775, 1104928158, 296696648, 298132798]


In [6]:
def MyCrack(vector, k):
    # Size = 128 * k + 6 (&&flag)
    s = "&&flag".encode()
    A = vector[0]
    B = vector[3]
    C = vector[2]
    D = vector[4]
    E = vector[5]
    F = vector[1]
    def G(X,Y,Z):
        return (X ^ (~Z | ~Y) ^ Z) & 0xFFFFFFFF
    def H(X,Y):
        return (X << Y | X >> (32 - Y)) & 0xFFFFFFFF
    X = [int((0xFFFFFFFE) * sin(i)) & 0xFFFFFFFF for i in range(256)]
    s_size = 128 * k + len(s)
    s += bytes([0x80])
    if len(s) % 128 > 120:
        while len(s) % 128 != 0: s += bytes(1)
    while len(s) % 128 < 120: s += bytes(1)
    # May pad to 120 bytes or 128 + 120 bytes
    print(b"After Padding: " + s)
    print("Length: " + str(len(s)), end="\n\n")
    # Add 8 Byte Message Length (in Bits)
    s += bytes.fromhex(hex(s_size * 8)[2:].rjust(16, '0'))
    print(b"Add Length in Bits: " + bytes.fromhex(hex(s_size * 8)[2:].rjust(16, '0')))
    print("Final Length: " + str(len(s)))
    print(b"Final Str: " + s)
    # Final Str => 128 Bytes * k
    for i, b in enumerate(s):
        # 32 Bytes a Block
        k, l = int(b), i & 0x1f
        A = (B + H(A + G(B,C,D) + X[k], l)) & 0xFFFFFFFF
        B = (C + H(B + G(C,D,E) + X[k], l)) & 0xFFFFFFFF
        C = (D + H(C + G(D,E,F) + X[k], l)) & 0xFFFFFFFF
        D = (E + H(D + G(E,F,A) + X[k], l)) & 0xFFFFFFFF
        E = (F + H(E + G(F,A,B) + X[k], l)) & 0xFFFFFFFF
        F = (A + H(F + G(A,B,C) + X[k], l)) & 0xFFFFFFFF
    return ''.join(map(lambda x : hex(x)[2:].rjust(8, '0'), [A, F, C, B, D, E]))

def generate_initial_padding(mes_size):
    print(mes_size)
    s = b""
    s += bytes([0x80])
    if (len(s)+mes_size) % 128 > 120:
        while (len(s)+mes_size) % 128 != 0: s += bytes(1)
    while (len(s)+mes_size) % 128 < 120: s += bytes(1)
    # May pad to 120 bytes or 128 + 120 bytes
    # Add 8 Byte Message Length (in Bits)
    s += bytes.fromhex(hex(mes_size * 8)[2:].rjust(16, '0'))
    print(b"Add Length in Bits: " + bytes.fromhex(hex(mes_size * 8)[2:].rjust(16, '0')))
    print("Final Length: " + str(len(s)))
#     print(b"Final Str: " + s)
    return s.hex()

In [10]:
# Test
tu = "HelloLEA"
tp = "pass"
tsession = "b12ce79e181631696d6d"
tmac = "2aa0ad1ce4a042081cf4bd7350d8f9082281cf98f4463652"

tv = []

for i in range(6):
    print(_seq[i] + ": " + tmac[i*8: (i+1)*8])
    tv.append(int(tmac[i*8: (i+1)*8], 16))

print(tv)

tl = len("&&".join([tu, tp, tsession]))
old_padding = generate_initial_padding(tl)
print(old_padding, end="\n\n\n\n")

ntmac = MyCrack(tv, 1)

print("\nnew t mac: " + ntmac)

print("Final Input: ")
print((b"&&".hex()).join([ntmac.encode().hex(), tsession.encode().hex()+old_padding, b"flag".hex()]))

A: 2aa0ad1c
F: e4a04208
C: 1cf4bd73
B: 50d8f908
D: 2281cf98
E: f4463652
[715173148, 3835707912, 485801331, 1356396808, 578932632, 4098242130]
36
b'Add Length in Bits: \x00\x00\x00\x00\x00\x00\x01 '
Final Length: 92
8000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000120



b'After Padding: &&flag\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Length: 120

b'Add Length in Bits: \x00\x00\x00\x00\x00\x00\x040'
Final Length: 128
b'Final Str: &&flag\x80\x00\x00\

In [8]:
server = b"&&flag\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x040"
my = b""

